In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.utilities import OpenWeatherMapAPIWrapper,WikipediaAPIWrapper
from langchain.agents import Tool
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document

In [3]:
from dotenv import load_dotenv
_ = load_dotenv()


In [4]:
weather_api = OpenWeatherMapAPIWrapper()  #https://home.openweathermap.org/api_keys

wiki_api = WikipediaAPIWrapper()


In [ ]:
embed = OpenAIEmbeddings(model='text-embedding-ada-002') 

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    

In [16]:
def wiki_search(query):
    data = wiki_api.load(query)
    split_docs = splitter.split_documents(data)
    vectorstore = FAISS.from_documents(split_docs, embed)
    vectorstore.save_local("faiss.index")
    return vectorstore.as_retriever()

In [8]:
def retriever_tool(query):
    retriever = wiki_search(query)
    results = retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results])


In [9]:
from langchain.llms import OpenAI
# from google.colab import userdata
# api_key = userdata.get('OPENAI_API')
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")  # or "gpt-4"


C:\Users\riina\AppData\Local\Temp\ipykernel_9964\89507056.py:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")  # or "gpt-4"


In [10]:
tools = [
    Tool(
    name="TravelInfoRetriever",
    func=retriever_tool,
    description=(
        "Use this tool to retrieve travel-related information about countries, cities, or landmarks. "
        "Ideal when the user asks something like: 'What is interesting in Paris?', or 'What is Tokyo known for?'."
    )
    ),

  Tool(
        name='Weather',
        func= weather_api.run,
        description=(
        "Use this tool to find **current or forecasted weather information** about a country, city, or travel destination. "
        "Ideal for questions like: 'What's the weather in Rome?', 'Is it rainy in Tokyo?', or 'How cold is it in Iceland in December?'. "
        "Only use this tool when the user is asking specifically about **weather** conditions. "
        "Do not use it for general travel info or sightseeing.")
  )
]

In [11]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
)

C:\Users\riina\AppData\Local\Temp\ipykernel_9964\1867655619.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  zero_shot_agent = initialize_agent(


In [15]:
zero_shot_agent.run("how is the weather in paris today?")



> Entering new AgentExecutor chain...
 I should use the Weather tool to find the current weather in Paris
Action: Weather
Action Input: "Paris"
Observation: In Paris, the current weather is as follows:
Detailed status: clear sky
Wind speed: 1.79 m/s, direction: 73°
Humidity: 63%
Temperature: 
  - Current: 15.92°C
  - High: 15.92°C
  - Low: 15.92°C
  - Feels like: 15.21°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought: I now know the final answer
Final Answer: The current weather in Paris is clear with a temperature of 15.92°C.

> Finished chain.


'The current weather in Paris is clear with a temperature of 15.92°C.'